In [1]:
import redis
import hashlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Ejemplo de predicción de modelo usando Redis



In [12]:
# Conectamos al servidor redis (correr el docker compose previamente, sino no funcionará)
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

In [40]:
# Obtención de datos para prueba

df_temp = pd.read_csv("./stroke_data.csv")

df_temp.sort_index(inplace=True)

# Sampleamos 100 valores al azar
df_temp = df_temp.sample(100)

test_values = df_temp.values.tolist()
# Conversión de valores a cadenas y hash
# Esto debería implementarse en el pipeline. Dado que los números de punto flotante pueden 
# presentar problemas debido a pequeñas variaciones, se podría considerar redondearlos.
string_values = [' '.join(map(str, sublist)) for sublist in test_values]
hashed_values = [hashlib.sha256(substring.encode()).hexdigest() for substring in string_values]  

# Obtención de las salidas del modelo desde Redis
# Estas salidas ya fueron procesadas en el pasado, por lo que consultamos Redis para obtenerlas.
model_output = [r.get(hash_key) for hash_key in hashed_values]

# Reemplazo de valores nulos con un valor predeterminado
# Esto es necesario porque, en caso de una nueva entrada que el modelo no haya visto durante 
# el procesamiento por lotes, necesitamos proporcionar una salida. Esta entrada podría ser
# encolada para su posterior procesamiento en lotes.
model_output = [value if value is not None else "mo stroke" for value in model_output]

Veamos la salida del modelo para diferentes entradas:

In [41]:
for index, test_value in enumerate(test_values):
    print(f"Para la entrada: {test_value}")
    
    print(f"El modelo predice que: {model_output[index]}")

Para la entrada: [-2.3582777733, -0.3130413977, -0.2200052704, -1.6011567771, -0.807834842, -1.9429587696, 1.0938967819, -0.6956791399, -0.2852221759, -0.0454076609, -0.9488378445, -0.3768526469, 3.6000698178, -0.7430314989, 1.1926566591, 2.1462396485, -0.3819536545, -0.596362755, -0.3331904251]
El modelo predice que: no_stroke
Para la entrada: [0.9229918901, 3.1944656754, -0.2200052704, 0.6245484604, 2.3163891445, 0.8022569804, -0.914163033, 1.4374442795, -0.2852221759, -0.0454076609, 1.0539208631, -0.3768526469, -0.2777723907, 1.3458379644, -0.8384642742, -0.4659311931, -0.3819536545, -0.596362755, -0.3331904251]
El modelo predice que: stroke
Para la entrada: [1.1508578389, -0.3130413977, -0.2200052704, 0.6245484604, -0.4750772576, 0.3447210221, -0.914163033, -0.6956791399, -0.2852221759, -0.0454076609, -0.9488378445, 2.6535570555, -0.2777723907, 1.3458379644, -0.8384642742, -0.4659311931, -0.3819536545, -0.596362755, -0.3331904251]
El modelo predice que: stroke
Para la entrada: [0.6